In [1]:
print('ready')

StatementMeta(SparkPool01, 2, 1, Finished, Available)

ready


In [2]:
from azureml.opendatasets import SanFranciscoSafety

data = SanFranciscoSafety()
df = data.to_spark_dataframe()
# Display 10 rows
display(df.limit(10))
df.printSchema()

StatementMeta(SparkPool01, 2, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 397a1e54-e1b7-475d-97d4-4101415c1139)

root
 |-- dataType: string (nullable = true)
 |-- dataSubtype: string (nullable = true)
 |-- dateTime: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- status: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- source: string (nullable = true)
 |-- extendedProperties: string (nullable = true)



In [3]:
from pyspark.sql.functions import lpad, col, month, year, dayofmonth
from pyspark.sql.types import datetime

StatementMeta(SparkPool01, 2, 3, Finished, Available)

In [4]:
df.columns

StatementMeta(SparkPool01, 2, 4, Finished, Available)

['dataType',
 'dataSubtype',
 'dateTime',
 'category',
 'subcategory',
 'status',
 'address',
 'latitude',
 'longitude',
 'source',
 'extendedProperties']

In [5]:
df_date = (df.select('dataType',
                    'dataSubtype',
                    'dateTime',
                    'category',
                    'subcategory',
                    'status',
                    'address',
                    'latitude',
                    'longitude',
                    'source',
                    'extendedProperties',
                    year('dateTime').alias('year'), 
                    lpad(month('dateTime'), 2, '0').alias('month'),
                    lpad(dayofmonth('dateTime'), 2, '0').alias('day_month')
                    )
            )

df_date.printSchema()

StatementMeta(SparkPool01, 2, 5, Finished, Available)

root
 |-- dataType: string (nullable = true)
 |-- dataSubtype: string (nullable = true)
 |-- dateTime: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- status: string (nullable = true)
 |-- address: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- source: string (nullable = true)
 |-- extendedProperties: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day_month: string (nullable = true)



In [9]:
(df_date.write
.format('csv')
.mode('overwrite')
.partitionBy('year', 'month', 'day_month')
.option('sep', '|')
.option('header', True)
.save('abfss://container01@dlsyntestmm.dfs.core.windows.net/sanfrancisco/csv'))

StatementMeta(SparkPool01, 2, 9, Finished, Available)

In [10]:
df_date.write.format('parquet').mode('overwrite').partitionBy('year', 'month', 'day_month').save('abfss://container01@dlsyntestmm.dfs.core.windows.net/sanfrancisco/parquet')

StatementMeta(SparkPool01, 2, 10, Finished, Available)

In [11]:
df_date.write.format('delta').mode('overwrite').partitionBy('year', 'month', 'day_month').save('abfss://container01@dlsyntestmm.dfs.core.windows.net/sanfrancisco/delta')

StatementMeta(SparkPool01, 2, 11, Finished, Available)